In [2]:
import pandas as pd
from pandas.core.interchange.dataframe_protocol import DataFrame

train_df = pd.read_csv("data/processed/train.csv")
val_df   = pd.read_csv("data/processed/val.csv")
test_df  = pd.read_csv("data/processed/test.csv")




In [3]:
train_df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,cat1,cat2,...,desc_unique_ratio,is_new,is_vintage,year_present,brand_bucket,brand_freq,is_popular_brand,cat1_brand_bucket,cat1_median_price,log_price
0,1097267,pyrex sunflower yellow 473 dish + lid,3,vintage & collectibles/housewares/glass,pyrex,20.0,0,pyrex sunflower yellow casserole dish and matc...,vintage & collectibles,housewares,...,0.780822,0,1,0,__freq__,6.656726,0,vintage & collectibles|__freq__,16.0,3.044522
1,1104122,nike hoodie,1,women/sweaters/hooded,nike,29.0,0,brand new with tags retail 50.00 price can cha...,women,sweaters,...,1.000000,1,0,0,__freq__,10.897553,1,women|__freq__,19.0,3.401197
2,694934,aeo pale pink jeggings size 8,2,"women/jeans/slim, skinny",american eagle,15.0,1,nwot!!! perfect color for spring! size 8 short...,women,jeans,...,0.931034,0,0,0,__freq__,9.492130,1,women|__freq__,19.0,2.772589
3,373236,9 vs no show thongs med nwt,1,women/underwear/g-strings & thongs,victoria's secret,45.0,1,6 vs no show thongs med nwt plus 3 large short...,women,underwear,...,0.912281,1,0,0,__freq__,10.779727,1,women|__freq__,19.0,3.828641
4,1118496,plush gray cardigan,2,women/sweaters/cardigan,croft & barrow,8.0,0,"croft & barrow , speckled gray & black cardiga...",women,sweaters,...,0.945946,1,0,0,__freq__,5.135798,0,women|__freq__,19.0,2.197225


In [4]:
TEXT_COLS= ['name', 'item_description']
CAT_COLS = ['cat1', 'cat2', 'cat3', 'brand_bucket', 'cat1_brand_bucket']
NUM_COLS = ['brand_freq', 'cat1_median_price', 'name_len', 'desc_len', 'desc_unique_ratio', 'is_new', 'is_vintage', 'year_present', 'is_popular_brand', 'item_condition_id']

In [ ]:
#Encoder v1: TF-IDF (name) + OHE (cat1) + numeric (brand_freq)”
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer

encoder = ColumnTransformer(
    transformers=[
        ("tf_name", TfidfVectorizer(ngram_range=(1,2),
                                    max_features=30000,
                                    min_df=2,
                                    max_df=0.98),
         "name"),
        ("tf_desc", TfidfVectorizer(ngram_range=(1,1),
                                    max_features=50000,
                                    min_df=5,
                                    max_df=0.95),
         "item_description"),
        ("ohe", OneHotEncoder(handle_unknown="ignore", dtype=np.int8), CAT_COLS),
        ("num", "passthrough", NUM_COLS),
    ],
    remainder="drop",
    sparse_threshold=0.0,          # keep output sparse
    verbose_feature_names_out=False # optional
)
X_train = encoder.fit_transform(train_df)
X_val = encoder.transform(val_df)

#Print X_train.shape, X_val.shape.
print(X_train.shape)
print(X_val.shape)


In [121]:
#Baseline Model

#Ridge
from sklearn.linear_model import Ridge
y_train = train_df['log_price'].values
y_val = val_df['log_price'].values

ridge_model = Ridge(alpha = 2.0, random_state=42)
ridge_model.fit(X_train, y_train)

y_val_pred = ridge_model.predict(X_val)

print("Model score on x_val")
ridge_model.score(X_train, y_train)

print("Model score on x_val")
ridge_model.score(X_val, y_val)

# 1. Prep targets
# •	Define y_train and y_val from train_df["log_price"] and val_df["log_price"].
# •	Quick check: qno NaNs, lengths match X_train, X_val.
#
# ⸻
#
# 2. Train Ridge (baseline model)
# •	Import Ridge from sklearn.linear_model.
# •	Initialize with alpha=2.0, random_state=42 (safe start).
# •	.fit(X_train, y_train) — confirm no memory issues.
#
# ⸻
#
# 3. Evaluate performance
# •	Predict on val: y_val_pred_log = model.predict(X_val).
# •	Compute metrics:
# •	RMSE_log and MAE_log on log_price.
# •	RMSE_price and MAE_price after np.expm1 transform (back to price).
# •	Print all 4 neatly.
#
# ⸻
#
# 4. Sanity check
# •	Take 5 random rows from val_df → show: name, cat1, true price vs predicted price.
# •	This will help you gut-check whether the model makes plausible guesses.

Model score on x_val
Model score on x_val


0.49869297620862774

In [108]:
y_val_pred = ridge_model.predict(X_val)

print(y_val_pred.shape)
print("Model score on x_train")
ridge_model.score(X_train, y_train)

print("Model score on x_val")
ridge_model.score(y_val_pred, y_val)

(203977,)
Model score on x_train
Model score on x_val


ValueError: Expected 2D array, got 1D array instead:
array=[2.70678612 2.83653664 2.97603053 ... 2.81608893 2.81054747 2.66129707].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [123]:
import numpy as np
import pandas as pd
from sklearn.metrics import root_mean_squared_error, mean_absolute_error

def evaluate_and_preview(y_true_log, y_pred_log, val_df, sample_n=5, seed=42, clip_display=False):
    """
    y_true_log, y_pred_log: arrays in LOG space (what your model predicted)
    val_df: the validation dataframe aligned with y_* (same order/length)
    sample_n: how many rows to print for a quick sanity check
    seed: fixed RNG for reproducible samples
    clip_display: if True, clip the previewed prices to [1, 99th pct] for readability (doesn't affect metrics)
    """

    # --- metrics: log space ---
    rmse_log = root_mean_squared_error(y_true_log, y_pred_log)
    mae_log  = mean_absolute_error(y_true_log, y_pred_log)

    # --- metrics: raw price space ---
    y_true_price = np.expm1(y_true_log)
    y_pred_price = np.expm1(y_pred_log)

    rmse_price = root_mean_squared_error(y_true_price, y_pred_price)
    mae_price  = mean_absolute_error(y_true_price, y_pred_price)

    print(f"RMSE_log  : {rmse_log:.4f}")
    print(f"MAE_log   : {mae_log:.4f}")
    print(f"RMSE_price: {rmse_price:,.2f}")
    print(f"MAE_price : {mae_price:,.2f}")

    # --- sample preview ---
    # rng = np.random.default_rng(seed)
    idx = np.random.choice(len(val_df), size=min(sample_n, len(val_df)), replace=False)
    preview = val_df.iloc[idx].copy()

    preview["true_price"] = y_true_price[idx]
    preview["pred_price"] = y_pred_price[idx]

    # Optional: clip for readability only (does not change metrics)
    if clip_display:
        lo, hi = 1.0, np.percentile(y_true_price, 99)  # simple global clip
        preview["true_price"] = preview["true_price"].clip(lo, hi)
        preview["pred_price"] = preview["pred_price"].clip(lo, hi)

    cols = ["name", "cat1", "true_price", "pred_price"]
    # Guard if a column is missing
    cols = [c for c in cols if c in preview.columns]
    print("\nSample predictions:")
    print(preview[cols].reset_index(drop=True))
    return preview

In [124]:
preview = evaluate_and_preview(y_val, y_val_pred, val_df, sample_n=10, seed=42, clip_display=False)

RMSE_log  : 0.5176
MAE_log   : 0.3917
RMSE_price: 28.61
MAE_price : 11.14

Sample predictions:
                                    name               cat1  true_price  \
0              all need is mascara shirt              women        15.0   
1                  bushnell range finder  sports & outdoors        49.0   
2                            winter hats              women         6.0   
3  jujube hello kitty peek a bow pacipod               kids        16.0   
4                       rit fabric tinte               kids         9.0   
5              holographic pink leggings              women        27.0   
6           the diggadilly racoon family               kids        14.0   
7                          alicia bundle               kids        31.0   
8     navy blue vineyard vines pull over              women        38.0   
9                         rae dunn tools               home        98.0   

   pred_price  
0    8.222844  
1   62.590442  
2   14.484654  
3   25.402785  

In [131]:
import numpy as np
import pandas as pd

# Convert predictions back to price scale
y_val_pred_price = np.expm1(y_val_pred)
y_val_true_price = np.expm1(y_val)

# Build DataFrame
prediction = pd.DataFrame({
    "cat1": val_df["cat1"].values,
    "name": val_df["name"].values,
    "true_price": y_val_true_price,
    "pred_price": y_val_pred_price
})

# Add row-level absolute error
prediction["abs_error"] = np.abs(prediction["true_price"] - prediction["pred_price"])
prediction["relative_error"] = prediction["pred_price"] / (prediction["true_price"] + 1e-6)
prediction = prediction.groupby(["cat1"]).agg({"abs_error": "mean", "relative_error": "mean"})
prediction.sort_values("abs_error", ascending=False, inplace=True)
prediction.head()

# electronics_price = prediction[prediction["cat1"] == "electronics"]
# electronics_price.sort_values("true_price", ascending=False, inplace=True)
# electronics_price.head()
# # count = pd.DataFrame(prediction.value_counts(["cat1"]))
# prediction = prediction.groupby(["cat1"]).agg({"abs_error": "median", "relative_error": "mean"})
#
# prediction.sort_values(by="abs_error", ascending=False, inplace=True)
# prediction.head()


#
# 3.	Group & aggregate
#
# •	Group by cat1 and compute:
# •	n = count
# •	mae = mean of abs_err
# •	median_ae = median absolute error
# •	p90_ae = 90th percentile absolute error
# •	mean_true and mean_pred (good for context)
# •	Put the result in a tidy table.
#
# 4.	Filter tiny groups
#
# •	Ignore super small categories so noise doesn’t dominate.
# •	With ~222k validation rows, a threshold of 500 (or 0.2% of val) is reasonable.
# •	Keep rows with n >= 500.
#
# 5.	Inspect
#
# •	Sort descending by mae and look at the top 10 worst categories.
# •	Also sort ascending and look at the top 10 best (to see where you’re already strong).
#
# 6.	Sanity cross-check (optional but pro)
#
# •	Compute the overall MAE in dollars from your full val set (you already have it ~$12.16).
# •	Verify that the count-weighted average of per-cat1 MAEs ≈ overall MAE.
# •	(This confirms your grouping math is aligned and you didn’t drop rows unexpectedly.)

,abs_error,relative_error
cat1,,
vintage & collectibles,15.402810,7723.850788
men,15.014850,8780.547643
unknown,12.358146,14219.489704
women,12.027782,13925.031178
sports & outdoors,11.580534,16022.956490
